# Projet de Data Mining (Web Scraping)

Le Jupyter Notebook suivant s'inscrit dans le projet de Data Mining sur la prédiction des matchs de football. Nous avons écrit ce code pour récupérer des informations supplémentaires qui, nous l'espérons, devraient nous permettre d'améliorer les prédictions de notre modèle. Ce code propose donc une technique de Web Scraping, en utilisant la librairie Selenium, permettant de récupérer les informations sur les notes Fifa des équipes et des joueurs de Ligue 1 (partie 1) et les notes provenant du journal L'Equipe des joueurs, entraîneurs et arbitres de Ligue 1 sur les matchs de championnat de Ligue 1 de la saison 2019-2020 (partie 2).

In [1]:
# Scrapping 
from selenium import webdriver # launch and initialise a browser
from selenium.webdriver.common.by import By # search using a specific parameter
from selenium.webdriver.support.ui import WebDriverWait # wait for page to load 
from selenium.webdriver.support import expected_conditions as EC #Specify what you are looking for on a specific page in order to determine that the webpage has loaded.
from selenium.common.exceptions import TimeoutException #Handling a timeout situation 

from time import sleep

In [2]:
option = webdriver.ChromeOptions()
option.add_argument("— incognito")
browser = webdriver.Chrome(executable_path= r'C:\Users\33679\chromedriver_win32\chromedriver.exe'
                           ,options=option)

In [3]:
def get_page(url):
    browser.maximize_window()
    sleep(1)
    browser.get(url)
    sleep(1)

## Récupération des notes des joueurs (Fifa)

https://www.fifaindex.com/fr/teams/?league=16

In [ ]:
url= 'https://www.fifaindex.com/fr/teams/?league=16'
get_page(url)

In [ ]:
equipe = []
attaque = []
milieu = []
defense = []
general = []
url = []

try : 
    for i in range(20):
        equipe.append(browser.find_elements_by_xpath("//td[@data-title='Nom']")[i].text)
        attaque.append(browser.find_elements_by_xpath("//td[@data-title='ATT']")[i].text)
        milieu.append(browser.find_elements_by_xpath("//td[@data-title='MIL']")[i].text)
        defense.append(browser.find_elements_by_xpath("//td[@data-title='DÉF']")[i].text)
        general.append(browser.find_elements_by_xpath("//td[@data-title='GEN']")[i].text)
        url.append(browser.find_elements_by_xpath("//td[@data-title='Nom']")[i].get_attribute("href"))
except : pass

equipe = pd.DataFrame(equipe, columns=["Equipe"])
attaque = pd.DataFrame(attaque, columns=["Attaque"])
milieu = pd.DataFrame(milieu, columns=["Milieu"])
defense = pd.DataFrame(defense, columns=["Défense"])
general = pd.DataFrame(general, columns=["Général"])

notes_equipes_L1 = pd.concat([equipe, attaque, milieu, defense, general], axis=1)
notes_equipes_L1.to_excel(
        "Données/Notes_FIFA/df_notes_equipe.xls", index=False)

Nous récupérons désormais les url de chaque équipe nous permettant d'accéder à la page recueillant les notes des joueurs par équipe. 

In [ ]:
browser.find_elements_by_class_name("link-team")[2].get_attribute("href")

In [ ]:
url_equipe = []
try : 
    for i in range(60):
        url_equipe.append(browser.find_elements_by_class_name("link-team")[i].get_attribute("href"))
except : pass

url_equipe = list(dict.fromkeys(url_equipe))

In [ ]:
for i in range(len(url_equipe)):
    print(url_equipe[i])

In [ ]:
def get_notes_joueurs():

    nom_joueur = []
    numero = []
    poste = []
    general = []
    potentiel = []

    try : 
        for i in range(40):
            nom_joueur.append(browser.find_elements_by_xpath("//td[@data-title='Nom']")[i].text)
            numero.append(browser.find_elements_by_xpath("//td[@data-title='Numéro']")[i].text)
            poste.append(browser.find_elements_by_xpath("//td[@data-title='Poste']")[i].text)
            general.append(browser.find_elements_by_xpath("//td[@data-title='GEN / POT']")[i].text[:2])
            potentiel.append(browser.find_elements_by_xpath("//td[@data-title='GEN / POT']")[i].text[-2:])

    except : pass

    nom_joueur = pd.DataFrame(nom_joueur, columns=['Nom'])
    numero = pd.DataFrame(numero, columns=["Numéro"])
    poste = pd.DataFrame(poste, columns=["Poste"])
    general = pd.DataFrame(general, columns=["Général"])
    potentiel = pd.DataFrame(potentiel, columns=["Potentiel"])

    notes_joueurs = pd.concat([nom_joueur, numero, poste, general, potentiel], axis=1)
    
    return notes_joueurs
    

In [ ]:
try : 
    for i in range(len(url_equipe)) :
        get_page(url_equipe[i])
        globals()["{}".format(equipe.loc[i][0])] = get_notes_joueurs()
        globals()["{}".format(equipe.loc[i][0])].to_excel("Données/Notes_joueurs//{}.xls".format(equipe.loc[i][0]),index=False)
except : pass

In [ ]:
PSG

## Récupération des notes des joueurs à partir du journal L'Equipe

### Récupération des notes individuelles des joueurs après chaque match (Journaux sportifs)

https://www.lequipe.fr/Football/les-notes/matchs/ligue-1/2019-2020/toutes-les-journees/

In [ ]:
url= 'https://www.lequipe.fr/Football/les-notes/matchs/ligue-1/2019-2020/toutes-les-journees/'
get_page(url)

On récupère l'url de chaque match où se trouve les notes des joueurs. Alternativement, il aurait été possible de cliquer directement dessus en utilisant les fonctionnalités de la librarie Selenium.

In [ ]:
url_match = []
try : 
    for i in range(1,281):
        url_match.append(browser.find_elements_by_class_name("list__item")[i].find_element_by_tag_name('a').get_attribute("href"))

except: pass

In [ ]:
len(url_match)

In [ ]:
 def get_notes_lequipe(url):
        
    home_team = []
    away_team = []
    score = []
    journee = []
    equipe_joueur = []
    joueur = []
    note_joueur = []


    for k in range(len(url)):
        try :
            get_page(url[k])

        except : pass 

        try : 

            for i in range(11) :
                journee.append(browser.find_element_by_class_name("SelectNav__selectWrapper").text.rsplit(',',2)[0].rsplit('\n',2)[1])
                home_team.append(browser.find_element_by_class_name("TeamScore").find_elements_by_tag_name('a')[0].text)
                away_team.append(browser.find_element_by_class_name("TeamScore").find_elements_by_tag_name('a')[2].text)
                score.append(browser.find_element_by_class_name("TeamScore").find_elements_by_tag_name('a')[1].text)
                equipe_joueur.append(browser.find_element_by_class_name("TeamScore").find_elements_by_tag_name('a')[0].text)
                joueur.append(browser.find_elements_by_class_name("fieldPlayer.fieldPlayer--positionned")[i].find_elements_by_tag_name('span')[1].text)
                note_joueur.append(browser.find_elements_by_class_name("fieldPlayer.fieldPlayer--positionned")[i].find_elements_by_tag_name('span')[0].text)

            for j in range(11,22):
                journee.append(browser.find_element_by_class_name("SelectNav__selectWrapper").text.rsplit(',',2)[0].rsplit('\n',2)[1])
                home_team.append(browser.find_element_by_class_name("TeamScore").find_elements_by_tag_name('a')[0].text)
                away_team.append(browser.find_element_by_class_name("TeamScore").find_elements_by_tag_name('a')[2].text)
                score.append(browser.find_element_by_class_name("TeamScore").find_elements_by_tag_name('a')[1].text)
                equipe_joueur.append(browser.find_element_by_class_name("TeamScore").find_elements_by_tag_name('a')[2].text)
                joueur.append(browser.find_elements_by_class_name("fieldPlayer.fieldPlayer--positionned")[j].find_elements_by_tag_name('span')[1].text)
                note_joueur.append(browser.find_elements_by_class_name("fieldPlayer.fieldPlayer--positionned")[j].find_elements_by_tag_name('span')[0].text)
        except : pass

    home_team = pd.DataFrame(home_team, columns=['HomeTeam'])
    away_team = pd.DataFrame(away_team, columns=['AwayTeam'])
    journee = pd.DataFrame(journee, columns=['Journée'])
    equipe_joueur = pd.DataFrame(equipe_joueur, columns= ['Equipe Joueur'])
    joueur = pd.DataFrame(joueur, columns=['Joueur'])
    note_joueur = pd.DataFrame(note_joueur, columns=["Note L'Equipe"])
    score = pd.DataFrame(score, columns = ['Score'])

    df_notes_lequipe = pd.concat([journee, home_team, away_team, score, equipe_joueur, joueur, note_joueur], axis = 1)
    df_notes_lequipe.to_excel("Données/Notes_Lequipe/df_notes_lequipe_1.xls",index=False)
    
    return df_notes_lequipe

In [ ]:
get_notes_lequipe(url_match)

Il faut compter environ 1h10min pour récupérer l'intégralité des notes des 10 matchs des 28 journées ayant eu lieu.

In [ ]:
df_notes_lequipe = pd.read_excel('Données/Notes_Lequipe/df_notes_lequipe.xls')

In [ ]:
df_notes_lequipe = df_notes_lequipe[df_notes_lequipe['Note L\'Equipe'] != '-']
df_notes_lequipe = df_notes_lequipe.reset_index(drop = True)
df_notes_lequipe["Note L\'Equipe"] = pd.to_numeric(df_notes_lequipe["Note L\'Equipe"])

In [1]:
df_notes_lequipe.groupby(['Joueur'])["Note L'Equipe"].mean().sort_values(ascending = False)

NameError: name 'df_notes_lequipe' is not defined

### Récupération des notes des arbitres et des entraîneurs après chaque match

https://www.lequipe.fr/Football/les-notes/matchs/ligue-1/2019-2020/toutes-les-journees/

In [ ]:
url= 'https://www.lequipe.fr/Football/les-notes/matchs/ligue-1/2019-2020/toutes-les-journees/'
get_page(url)

In [ ]:
url_match = []
try : 
    for i in range(1,281):
        url_match.append(browser.find_elements_by_class_name("list__item")[i].find_element_by_tag_name('a').get_attribute("href"))

except: pass

In [ ]:
   def get_notes_lequipe_coach(url):
    
    home_team = []
    away_team = []
    score = []
    journee = []
    equipe_entraineur_domicile = []
    entraineur_domicile = []
    note_entraineur_domicile = []
    equipe_entraineur_exterieur = []
    entraineur_exterieur = []
    note_entraineur_exterieur = []
    arbitre = []
    note_arbitre = []


    for k in range(len(url)):
        try:
            get_page(url[k])

        except:
            pass

        try:
            journee.append(browser.find_element_by_class_name(
                "SelectNav__selectWrapper").text.rsplit(',', 2)[0].rsplit('\n', 2)[1])
            home_team.append(browser.find_element_by_class_name(
                "TeamScore").find_elements_by_tag_name('a')[0].text)
            away_team.append(browser.find_element_by_class_name(
                "TeamScore").find_elements_by_tag_name('a')[2].text)
            score.append(browser.find_element_by_class_name(
                "TeamScore").find_elements_by_tag_name('a')[1].text)
            entraineur_domicile.append(browser.find_elements_by_class_name(
                "RatingsArchivesPersonRating__personName")[0].text)
            entraineur_exterieur.append(browser.find_elements_by_class_name(
                "RatingsArchivesPersonRating__personName")[1].text)
            arbitre.append(browser.find_elements_by_class_name(
                "RatingsArchivesPersonRating__personName")[2].text)
            equipe_entraineur_domicile.append(browser.find_elements_by_class_name(
                "RatingsArchivesPersonRating__personInfo")[0].text)
            equipe_entraineur_exterieur.append(browser.find_elements_by_class_name(
                "RatingsArchivesPersonRating__personInfo")[1].text)
            note_arbitre.append(browser.find_elements_by_class_name(
                "RatingsArchivesPersonRating__ratingValue")[2].text)
            note_entraineur_domicile.append(browser.find_elements_by_class_name(
                "RatingsArchivesPersonRating__ratingValue")[0].text)
            note_entraineur_exterieur.append(browser.find_elements_by_class_name(
                "RatingsArchivesPersonRating__ratingValue")[1].text)

        except:
            pass

    home_team = pd.DataFrame(home_team, columns=['HomeTeam'])
    away_team = pd.DataFrame(away_team, columns=['AwayTeam'])
    journee = pd.DataFrame(journee, columns=['Journée'])
    score = pd.DataFrame(score, columns=['Score'])
    entraineur_domicile = pd.DataFrame(
        entraineur_domicile, columns=['Entraineur Domicile'])
    equipe_entraineur_domicile = pd.DataFrame(
        equipe_entraineur_domicile, columns=['Equipe Coach Dom'])
    note_entraineur_domicile = pd.DataFrame(
        note_entraineur_domicile, columns=["Note Coach Dom"])
    entraineur_exterieur = pd.DataFrame(
        entraineur_exterieur, columns=['Entraineur Exterieur'])
    equipe_entraineur_exterieur = pd.DataFrame(
        equipe_entraineur_exterieur, columns=['Equipe Coach Ext'])
    note_entraineur_exterieur = pd.DataFrame(
        note_entraineur_exterieur, columns=["Note Coach Ext"])
    arbitre = pd.DataFrame(arbitre, columns=['Arbitre'])
    note_arbitre = pd.DataFrame(note_arbitre, columns=['Note Arbitre'])


    df_notes_lequipe_coach = pd.concat([journee, home_team, away_team, score, entraineur_domicile,
                                  equipe_entraineur_domicile, note_entraineur_domicile, entraineur_exterieur, 
                                  equipe_entraineur_exterieur, note_entraineur_exterieur, arbitre, note_arbitre], axis=1)
    df_notes_lequipe_coach.to_excel(
        "Données/Notes_Lequipe/df_notes_lequipe_coach.xls", index=False)
    
    return df_notes_lequipe_coach

In [ ]:
get_notes_lequipe_coach(url_match)

In [ ]:
df_notes_lequipe_coach = pd.read_excel('Données/Notes_Lequipe/df_notes_lequipe_coach.xls')

In [ ]:
df_notes_lequipe_coach.head(10)